# Air Quality Data Analysis


## Metadata Dataset
### Deskripsi
This data set includes hourly air pollutants data from 12 nationally-controlled air-quality monitoring sites. The air-quality data are from the Beijing Municipal Environmental Monitoring Center. The meteorological data in each air-quality site are matched with the nearest weather station from the China Meteorological Administration. The time period is from March 1st, 2013 to February 28th, 2017. Missing data are denoted as NA.
### Column

* ***No***: *Row number*
* ***Year***: *Year of data in this row*
* ***Month***: *Month of data in this row*
* ***Day***: *Day of data in this row*
* ***Hour***: *Hour of data in this row*
* ***PM2.5***: *PM2.5 concentration ($\mu g /m^3$)*
* ***PM10***: *PM10 concentration ($\mu g /m^3$))*
* ***SO2***: *SO2 concentration ($\mu g /m^3$)*
* ***NO2***: *NO2 concentration ($\mu g /m^3$)*
* ***CO***: *CO concentration ($\mu g /m^3$)*
* ***O3***: *O3 concentration ($\mu g /m^3$)*
* ***TEMP***: *Temperature ($\degree C$)*
* ***PRES***: *Pressure ($hPa$)*
* ***DEWP***: *Dew point temperature ($\degree C$)*
* ***RAIN***: *Precipitation ($mm$)*
* ***wd***: *Wind direction*
* ***WSPM***: *Wind speed ($m/s$)*
* ***station***: *Name of the air-quality monitoring site*

Reference: [UCI Machine Learning Repository| Beijing Multi-Site Air Quality](https://archive.ics.uci.edu/dataset/501/beijing+multi+site+air+quality+data)

## Pertanyaan Bisnis
Berdasarkan deskripsi dari dataset, dirumuskan pertanyaan bisnis sebagai berikut:
 1.  Bagaimana hubungan arah angin terhadap kualitas udara di masing-masing *station* selama periode 2013-2017?
 2.  Bagaimana hubungan antara musim dengan kualitas udara di berbagai *station* pada tahun 2013-2017?
 3.  Kapan kualitas udara berada dalam kondisi terburuk di masing-masing *station* selama periode 2013-2017?
 4.  Bagaimana perbandingan kualitas udara pada masing-masing *station* perbandingan selama periode 2013-2017?
 5.  Bagaimana hubungan hujan terhadap kualias udara di masing-masing *station* selama periode 2013-2017?
 6.  Bagaimana tren kualitas udara di masing-masing *station* selama periode 2013-2017? 

## Batasan
Berdasarkan pertanyaan bisnis, beberapa batasan dalam analisis berikut:
1. Kualitas udara ditentukan oleh nilai konsentrasi polutan PM2.5.
2. Pada pertanyaan bisnis ke-1 analisa dilakukan dengan data perjam, sedangkan data lainnya menggunakan data yang telah dirata-ratakan perhari.
3. Musim yang digunakan dalam analisi berdasarkan musim yang ada di China, dengan pembagian musim berdasarkan bulan sebagai berikut:
   - Musim Dingin (Desember - Februari)
   - Musim Semi (Maret-Mei)
   - Musim Panas (Juni - Agustus)
   - Musim Gugur (September - November)
4. Standar kualitas udara yang digunakan adalah *China Air Quality Guidelines* dan *National Ambient Air Quality Standards (NAAQS)*.
5. Fitur yang digunakan adalah Year, Month, Day, Hour, PM2.5, RAIN, wd, wspm 

### China Air Quality Guidelines
Standard kualitas udara China yang terbaru dikeluarkan pada tahun 2012. Pada *guidelines* ini, mengatur tentang konsentrasi berbagai polutan pada dua kelas daerah.
1. **Class 1**: daerah khusus seperti taman nasional.
2. **Class 2**, daerah lainnya seperti perkotaan atau daerah industri.

Table 1. Standar Maksimum Kualitas Udara berdasarkan Polutan PM2.5
|Pollutant|Averageing time|Class 1|Class 2|
|---|---|---|---|
|$PM_{2.5}$ ($\mu g/m^3$)|annual|15|35|
||24 hours|35|75|

Reference: [ China: Air Quality Standards](https://www.transportpolicy.net/standard/china-air-quality-standards/)

### National Ambient Air Quality Standards (NAAQS)  

NAAQS merupakan standar yang dikeluarkan oleh United States Environmental Protection Agency (EPA). Pada Gambar 1, merupakan klasifikasi berdasarkan konsentrasi PM2.5.

![AQI Guidelines Chart!](asset/2236_WHO_Guidlines_Chart_UPDATE_AQI_2024.webp)

Gambar 1. Standar Kualitas Udara US 2024 ([reference](https://www.iqair.com/id/newsroom/what-is-aqi))

In [2]:
import os
import json
import pandas as pd
from rich import print as rprint
from rich.console import Console
from rich.table import Table
import plotly.express as px
from typing import Tuple, Annotated, Dict
import numpy as np

In [3]:
ROOT_PATH = os.getcwd()
ROOT_PATH

'/home/pradanaend/Documents/Project/air-quality-analysis'

### Utils


In [4]:
def missing_value_summary(df: Dict[Annotated[str, "station name"], pd.DataFrame]):
    missing_value_summary = {}
    for station, dataframe in df.items():
        missing_data_sum = dataframe.isna().sum()
        missing_percent = ((missing_data_sum/len(dataframe))*100)
        station_summary = pd.DataFrame({
            '#missing':missing_data_sum,
            "%missing":missing_percent
        })
        station_summary = station_summary[station_summary['#missing']>0]
        missing_value_summary[station] = station_summary
    return missing_value_summary

def display_consolidated_missing_summary(missing_summary):
    console = Console()
    table = Table(title="Missing Values Across All Stations", show_header=True, header_style="bold white")
    
    table.add_column("Station", style="cyan")
    table.add_column("Column Name", style="yellow")
    table.add_column("Missing Count", justify="right")
    table.add_column("Missing Percentage (%)", justify="right")
    
    for station, summary_df in missing_summary.items():
        if summary_df.empty:
            table.add_row(station, "No missing values", "", "")
        else:
            for col_name, row in summary_df.iterrows():
                table.add_row(
                    station,
                    col_name,
                    str(row["#missing"]),
                    f"{row['%missing']:.2f}"
                )
    
    console.print(table)

In [5]:
def iqr_outlier_threshold(
    df_series: pd.Series,
) -> Annotated[Tuple[float, float], "(Lower bound, Upper bound)"]:
    q1 = df_series.quantile(0.25)
    q3 = df_series.quantile(0.75)
    IQR = q3 - q1
    rprint(f"Q1 : {q1}\nQ3 : {q3}\nIQR : {IQR}")
    lower_bound = q1 - 1.5 * IQR
    upper_bound = q3 + 1.5 * IQR

    return lower_bound, upper_bound


def get_outlier_index(
    df_series: pd.Series,
) -> Annotated[
    Tuple[np.ndarray, np.ndarray], "(Lower outlier array, Upper outlier array)"
    
]:
    threshold_lower, threshold_upper = iqr_outlier_threshold(df_series)
    lower_outlier_array = np.where(df_series <= threshold_lower)[0]
    upper_outlier_array = np.where(df_series >= threshold_upper)[0]
    
    return lower_outlier_array, upper_outlier_array

In [6]:
with open('data/beijing.geojson', encoding='utf-8-sig') as f:
    beijing_geojson = json.load(f)

## Data Wrangling

### Load Data

In [19]:
df_aotizhongxin = pd.read_csv("data/PRSA_Data_Aotizhongxin_20130301-20170228.csv")
df_changping = pd.read_csv("data/PRSA_Data_Changping_20130301-20170228.csv")
df_dingling = pd.read_csv("data/PRSA_Data_Dingling_20130301-20170228.csv")
df_dongsi = pd.read_csv("data/PRSA_Data_Dongsi_20130301-20170228.csv")
df_guanyuan = pd.read_csv("data/PRSA_Data_Guanyuan_20130301-20170228.csv")
df_gucheng = pd.read_csv("data/PRSA_Data_Gucheng_20130301-20170228.csv")
df_huairou = pd.read_csv("data/PRSA_Data_Huairou_20130301-20170228.csv")
df_nongzhanguan = pd.read_csv("data/PRSA_Data_Nongzhanguan_20130301-20170228.csv")
df_shunyi = pd.read_csv("data/PRSA_Data_Shunyi_20130301-20170228.csv")
df_tiantan = pd.read_csv("data/PRSA_Data_Tiantan_20130301-20170228.csv")
df_wanliu = pd.read_csv("data/PRSA_Data_Wanliu_20130301-20170228.csv")
df_wanshouxigong = pd.read_csv("data/PRSA_Data_Wanshouxigong_20130301-20170228.csv")



In [40]:
df_aotizhongxin

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400.0,95.0,12.5,1013.5,-16.2,0.0,NW,2.4,Aotizhongxin
35060,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500.0,81.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Aotizhongxin
35061,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700.0,58.0,10.8,1014.2,-13.3,0.0,NW,1.1,Aotizhongxin
35062,35063,2017,2,28,22,21.0,44.0,12.0,87.0,700.0,35.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Aotizhongxin


In [20]:
df = {
    'Aotizhongxin': df_aotizhongxin,
    'Changping': df_changping,
    'Dingling': df_dingling,
    'Dongsi': df_dongsi,
    'Guanyuan': df_guanyuan,
    'Gucheng': df_gucheng,
    'Huairou': df_huairou,
    'Nongzhanguan': df_nongzhanguan,
    'Shunyi': df_shunyi,
    'Tiantan': df_tiantan,
    'Wanliu': df_wanliu,
    'Wanshouxigong': df_wanshouxigong
}

### Data Cleaning

#### Data condition

##### Missing Value

In [44]:
missing_summary = missing_value_summary(df)


display_consolidated_missing_summary(missing_summary)

                   Missing Values Across All Stations                   
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Station       ┃ Column Name ┃ Missing Count ┃ Missing Percentage (%) ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Aotizhongxin  │ PM2.5       │         925.0 │                   2.64 │
│ Aotizhongxin  │ PM10        │         718.0 │                   2.05 │
│ Aotizhongxin  │ SO2         │         935.0 │                   2.67 │
│ Aotizhongxin  │ NO2         │        1023.0 │                   2.92 │
│ Aotizhongxin  │ CO          │        1776.0 │                   5.07 │
│ Aotizhongxin  │ O3          │        1719.0 │                   4.90 │
│ Aotizhongxin  │ TEMP        │          20.0 │                   0.06 │
│ Aotizhongxin  │ PRES        │          20.0 │                   0.06 │
│ Aotizhongxin  │ DEWP        │          20.0 │                   0.06 │
│ Aotizhongxin  │ RAIN        │          20.0 │                   0.06 │
│ Aotizhongxin  │ wd          │          81.0 │                   0.23 │
│ Aotizhongxin  │ WSPM        │          14.0 │                   0.04 │
│ Changping     │ PM2.5       │         774.0 │                   2.21 │
│ Changping     │ PM10        │         582.0 │                   1.66 │
│ Changping     │ SO2         │         628.0 │                   1.79 │
│ Changping     │ NO2         │         667.0 │                   1.90 │
│ Changping     │ CO          │        1521.0 │                   4.34 │
│ Changping     │ O3          │         604.0 │                   1.72 │
│ Changping     │ TEMP        │          53.0 │                   0.15 │
│ Changping     │ PRES        │          50.0 │                   0.14 │
│ Changping     │ DEWP        │          53.0 │                   0.15 │
│ Changping     │ RAIN        │          51.0 │                   0.15 │
│ Changping     │ wd          │         140.0 │                   0.40 │
│ Changping     │ WSPM        │          43.0 │                   0.12 │
│ Dingling      │ PM2.5       │         779.0 │                   2.22 │
│ Dingling      │ PM10        │         656.0 │                   1.87 │
│ Dingling      │ SO2         │         730.0 │                   2.08 │
│ Dingling      │ NO2         │        1234.0 │                   3.52 │
│ Dingling      │ CO          │        2012.0 │                   5.74 │
│ Dingling      │ O3          │        1214.0 │                   3.46 │
│ Dingling      │ TEMP        │          53.0 │                   0.15 │
│ Dingling      │ PRES        │          50.0 │                   0.14 │
│ Dingling      │ DEWP        │          53.0 │                   0.15 │
│ Dingling      │ RAIN        │          51.0 │                   0.15 │
│ Dingling      │ wd          │         140.0 │                   0.40 │
│ Dingling      │ WSPM        │          43.0 │                   0.12 │
│ Dongsi        │ PM2.5       │         750.0 │                   2.14 │
│ Dongsi        │ PM10        │         553.0 │                   1.58 │
│ Dongsi        │ SO2         │         663.0 │                   1.89 │
│ Dongsi        │ NO2         │        1601.0 │                   4.57 │
│ Dongsi        │ CO          │        3197.0 │                   9.12 │
│ Dongsi        │ O3          │         664.0 │                   1.89 │
│ Dongsi        │ TEMP        │          20.0 │                   0.06 │
│ Dongsi        │ PRES        │          20.0 │                   0.06 │
│ Dongsi        │ DEWP        │          20.0 │                   0.06 │
│ Dongsi        │ RAIN        │          20.0 │                   0.06 │
│ Dongsi        │ wd          │          78.0 │                   0.22 │
│ Dongsi        │ WSPM        │          14.0 │                   0.04 │
│ Guanyuan      │ PM2.5       │         616.0 │                   1.76 │
│ Guanyuan      │ PM10        │         429.0 │                   1.22 │
│ Guanyuan      │ SO2         │         474.0 │           